# Logging

##### author: Alex Sherman | alsherman@deloitte.com
    
#### Sources:
- https://docs.python.org/2/howto/logging.html
- http://docs.python-guide.org/en/latest/writing/logging/

In [1]:
from IPython.core.display import display, HTML
sqlalchemy_url = 'http://docs.python-guide.org/en/latest/writing/logging/'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(sqlalchemy_url)
HTML(iframe)

With logging you can control which messages to display (filtering out unimportant messages with logging levels), when to display messages, how to format messages (logging format), and where to output messages (sending different messages to differnt locations with logging handlers)

##### Logging serves two purposes:
- Diagnostic logging records events related to the application’s operation. If a user calls in to report an error, for example, the logs can be searched for context.
- Audit logging records events for business analysis. A user’s transactions can be extracted and combined with other user details for reports or to optimize a business goal.

The are numerous ways to set up a logging file. We will use a separate file named logging_config.ini to configure all of the logging settings. Then we will import these settings into a instantiated logger in our python script.

### Example config.ini


- Loggers expose the interface that application code directly uses.
- Handlers send the log records (created by loggers) to the appropriate destination.
- Filters provide a finer grained facility for determining which log records to output.
- Formatters specify the layout of log records in the final output.

##### [loggers]
List all of the loggers used in the application. The root logger is the default. 

##### [handlers]
List all of the handlers used in the application. The stream_handler is the default. 

##### [formatters]
Defines the format of the logging output. As an example, the following format outputs the time, name, logging level, and logging message
- format=%(asctime)s %(name)-12s %(levelname)-8s %(message)s

##### [logger_root]
This is an example of one logger. Additional (custom) loggers may be added below using the naming convention of [logger_new_logger_name]. 
 
The logger defines the logging information such as the level (e.g. DEBUG) and associated handlers. Often, there may be a separate logger/s for each python module. These new loggers may be specified to define what output to retain and where to store the output. 

##### [handler_stream_handler]
This is an example of one handler. Additional (custom) handlers may be listed below using the namin convention of [formatter_formatter]

Defines the handler information such as the level, format, and args. Handlers are often set to define where to store logging output (e.g. flatfile, database, etc.). 

Source: https://docs.python.org/2/howto/logging.html

#### Read in the logging_config.ini file

Note: The logging_config.ini is separate files (in the lesson directory). The code above is an example of the code in the logging file, but not the one we are importing below.

In [2]:
import logging
from logging.config import fileConfig

# read in the logging configurations
fileConfig('logging_config.ini')

# instantiate the logger
logger = logging.getLogger()

# send a logging message to the application
logger.debug('Example logging message')

2017-11-26 17:31:38,187 root         DEBUG    Example logging message


## Add a new logger and handler with a different logging level

#### In version 2 of the logging_config file we update add a new logger and handler to send output to a local file

NOTE: only the sections with modifications are displayed. View logging_config_v2.ini to see the full config file specification.

In [45]:
# test new logger - output available in text file specified in the config file.
fileConfig('logging_config_v2.ini')
logger = logging.getLogger('filetest')
logger.info('Example logging message')  # change logger.debug to logger.info

2017-11-26 17:37:05,745 filetest     INFO     Example logging message


In [46]:
# no output displays when we continue to use logger.debug, because we changed the logger
# level from debug to info in the config file. INFO is a higher level that debug
fileConfig('logging_config_v2.ini')
logger = logging.getLogger('filetest')
logger.debug('Example logging')

## Loggers

### Loggers purpose
Logger objects have a threefold job. First, they expose several methods to application code so that applications can log messages at runtime. Second, logger objects determine which log messages to act upon based upon severity (the default filtering facility) or filter objects. Third, logger objects pass along relevant log messages to all interested log handlers.

### Logging Levels:
With logging levels we can set for different message to occur based off the severity of an event. For instance, we may only turn on debugging (DEBUG LEVEL) while fixing code (in place of adding/ removing print statements), always send a confirmation message when a record is inserted into a database (INFO level), but only use logging error messages (ERROR LEVEL) when a exception event occurs

##### Logging Levels
- DEBUG (lowest)
- INFO
- WARNING
- ERROR
- CRITICAL (highest)

#### Example logging code from a project, using multiple logging levels
**NOTE:** This code will not work. It is just provided as an example. The try except block is used to avoid a NameError.

In [21]:
try:
    # extract text from documents from the input folder and insert into FITARA database
    for ind, path in enumerate(yield_document_paths(process.input)):
        # files are moved from the input folder to the in progress folder to denote the start of the process
        process.move_file(path, new_root='in_progress')

        if ind == 0:
            logger.info('New documents in queue - update starting')

        if DatabaseConnection.determine_if_filename_in_db(filename=path.filename):
            # provide user a message when adding a file with an existing name in the db
            # timestamps are added to file names to resolve issue of multile files with the same name
            # in the SUCCESS and FAILURE folders (e.g. two different files with the name SOW.docx)
            logger.warning('A file with the same name exists: {}'.format(path.filename))

        # The text extraction code only works for docx files (not pdf or doc)
        # If the original file is not a docx then the file must be converted or the file fails.
        # Converted files are stored in a separate folder
        # The document in the converted files folder is used to extract the text, while the original file moves through
        # the process so that after the code completed, users have access to the original file in the success folder.
        if path.incorrect_file_type() and not path.file_already_converted(process.converted_files_path):
            msg = "Document failed - please convert (open file and save as docx): {}".format(path.filename)
            logger.warning(msg)
            process.move_file(path, new_root='failed')
            continue
except:
    pass

### Logging Handlers
Handler objects are responsible for dispatching the appropriate log messages (based on the log messages’ severity) to the handler’s specified destination. Logger objects can add zero or more handler objects to themselves with an addHandler() method. As an example scenario, an application may want to send all log messages to a log file, all log messages of error or higher to stdout, and all messages of critical to an email address. This scenario requires three individual handlers where each handler is responsible for sending messages of a specific severity to a specific location.

It is, of course, possible to log messages to different destinations. Support is included in the package for writing log messages to files, HTTP GET/POST locations, email via SMTP, generic sockets, or OS-specific logging mechanisms such as syslog or the Windows NT event log. Destinations are served by handler classes. You can create your own log destination class if you have special requirements not met by any of the built-in handler classes.

 if no destination is set; and if one is not set, they will set a destination of the console (sys.stderr) 
 
The standard library includes quite a few handler types (see Useful Handlers); the tutorials use mainly StreamHandler and FileHandler in its examples. Other handlers are listed in the website below.

In [49]:
sqlalchemy_url = 'https://docs.python.org/2/howto/logging.html#useful-handlers'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(sqlalchemy_url)
HTML(iframe)

### Logging Formatters

Formatter objects configure the final order, structure, and contents of the log message. 

In [58]:
# no output displays when we continue to use logger.debug, because we changed the logger
# level from debug to info in the config file. INFO is a higher level that debug
fileConfig('logging_config_v3.ini')
logger = logging.getLogger()
logger.debug('Example logging')

DEBUG                          Example logging


### Logging Flow

# TODO: adding logging flow image

### Python Exceptions

Even if a statement or expression is syntactically correct, it may cause an error when an attempt is made to execute it. Errors detected during execution are called exceptions and are not unconditionally fatal: 

#### The try statement works as follows.

- First, the try clause (the statement(s) between the try and except keywords) is executed.
- If no exception occurs, the except clause is skipped and execution of the try statement is finished.
- If an exception occurs during execution of the try clause, the rest of the clause is skipped. Then if its type matches the exception named after the except keyword, the except clause is executed, and then execution continues after the try statement.
- If an exception occurs which does not match the exception named in the except clause, it is passed on to outer try statements; if no handler is found, it is an unhandled exception and execution stops with a message as shown above.

try:
    variable_does_not_exist
except Exception as e:
    logger.error('Please create the variable before using it')

In [51]:
try:
    variable_does_not_exist
except Exception as e:
    logger.error('Please create the variable before using it')

2017-11-26 18:32:44,556 filetest     ERROR    Please create the variable before using it


In [50]:
try:
    variable_does_not_exist
except Exception as e:
    logger.error('Please create the variable before using it'
                 , exc_info=True)

2017-11-26 18:31:52,760 filetest     ERROR    Please create the variable before using it
Traceback (most recent call last):
  File "<ipython-input-50-dc7c61b494f9>", line 2, in <module>
    variable_does_not_exist
NameError: name 'variable_does_not_exist' is not defined


### Error Catching
- It is a better practice to be explicit in the exceptions you are catching
- here we catch the NameError, but change the actual error to a SyntaxError
- by forgetting to add parenthesis after our print statement
- thus, we do not catch the error

In [54]:
try:
    print variable_does_not_exist
except NameError as e:
    logger.error('Please create the variable before using it'
                 , exc_info=True)

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-54-c30b32a36807>, line 2)

In [52]:
sqlalchemy_url = 'https://docs.python.org/3/tutorial/errors.html'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(sqlalchemy_url)
HTML(iframe)